### Load in the training and test datasets

In [1]:
import pandas as pd
import numpy as np

# Use read_excel and use participant_id as the index
train_cat = pd.read_excel("../../widsdatathon2025/TRAIN_NEW/TRAIN_CATEGORICAL_METADATA_new.xlsx")
train_func = pd.read_csv("../../widsdatathon2025/TRAIN_NEW/"
                         "TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson.csv")
train_quant = pd.read_excel("../../widsdatathon2025/TRAIN_NEW/TRAIN_QUANTITATIVE_METADATA_new.xlsx")
train_soln = pd.read_excel("../../widsdatathon2025/TRAIN_NEW/TRAINING_SOLUTIONS.xlsx")

# TEST
test_cat = pd.read_excel("../../widsdatathon2025/TEST/TEST_CATEGORICAL.xlsx")
test_func = pd.read_csv("../../widsdatathon2025/TEST/TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv")
test_quant = pd.read_excel("../../widsdatathon2025/TEST/TEST_QUANTITATIVE_METADATA.xlsx")
solutions_path = "../../widsdatathon2025/TEST/TEST_SOLUTIONS.xlsx"

In [2]:
# Set index for merging
train_cat.set_index("participant_id", inplace=True)
train_quant.set_index("participant_id", inplace=True)
train_func.set_index("participant_id", inplace=True)
train_soln.set_index("participant_id", inplace=True)

# TEST
test_cat.set_index("participant_id", inplace=True)
test_quant.set_index("participant_id", inplace=True)
test_func.set_index("participant_id", inplace=True)

In [3]:
test_func.head()
# train_func.head()
# train_quant.head()
# train_soln.head()

,0throw_1thcolumn,0throw_2thcolumn,0throw_3thcolumn,0throw_4thcolumn,0throw_5thcolumn,0throw_6thcolumn,0throw_7thcolumn,0throw_8thcolumn,0throw_9thcolumn,0throw_10thcolumn,...,195throw_196thcolumn,195throw_197thcolumn,195throw_198thcolumn,195throw_199thcolumn,196throw_197thcolumn,196throw_198thcolumn,196throw_199thcolumn,197throw_198thcolumn,197throw_199thcolumn,198throw_199thcolumn
participant_id,,,,,,,,,,,,,,,,,,,,,
Cfwaf5FX7jWK,0.548480,0.713607,0.557319,0.524369,0.693364,0.770032,0.724406,0.390118,0.547912,0.470220,...,0.080423,-0.054581,-0.088163,-0.028574,0.444847,0.350149,-0.012601,0.665750,0.560565,0.555732
vhGrzmvA3Hjq,0.427740,0.363022,0.402862,0.363003,0.534558,0.345347,0.409471,0.303328,0.402515,0.275052,...,0.198009,-0.000724,0.083122,0.033043,0.687497,0.306229,0.717485,0.461809,0.559632,0.350027
ULliyEXjy4OV,0.139572,0.390106,-0.087041,0.196852,0.088148,0.023843,0.381782,0.068979,0.377488,0.301871,...,0.051319,0.023630,-0.056819,0.117396,0.576086,0.517831,0.527044,0.605038,0.609856,0.750987
LZfeAb1xMtql,0.133561,0.778326,0.416355,0.471840,0.568460,0.633660,0.501113,0.345461,0.467943,0.417308,...,0.046183,-0.238962,0.121868,-0.260970,0.646818,0.594902,0.608156,0.595459,0.683189,0.542296
EnFOUv0YK1RG,0.126699,0.575446,0.509422,0.363193,0.427544,0.449924,0.451796,0.223927,0.298248,0.304429,...,0.315734,0.002234,0.290791,0.344149,0.480214,0.539824,0.447322,0.293088,0.148529,0.539823


## PCA for the functional connectome matrix to reduce dimension

In [4]:
from sklearn.decomposition import PCA

# Select only fMRI columns
fmri_data = train_func.drop(columns=['participant_id'], errors='ignore')
fmri_data_test = test_func.drop(columns=['participant_id'], errors='ignore')

# Apply PCA to reduce to top N components (e.g. 50)
pca = PCA(n_components=50)
fmri_pca = pca.fit_transform(fmri_data)
fmri_pca_test = pca.fit_transform(fmri_data_test)

train_func_pca = pd.DataFrame(fmri_pca, index=train_func.index,
                           columns=[f'fmri_pca_{i}' for i in range(1, 51)])
test_func_pca = pd.DataFrame(fmri_pca_test, index=test_func.index,
                           columns=[f'fmri_pca_{i}' for i in range(1, 51)])
train_func_pca.head()

,fmri_pca_1,fmri_pca_2,fmri_pca_3,fmri_pca_4,fmri_pca_5,fmri_pca_6,fmri_pca_7,fmri_pca_8,fmri_pca_9,fmri_pca_10,...,fmri_pca_41,fmri_pca_42,fmri_pca_43,fmri_pca_44,fmri_pca_45,fmri_pca_46,fmri_pca_47,fmri_pca_48,fmri_pca_49,fmri_pca_50
participant_id,,,,,,,,,,,,,,,,,,,,,
70z8Q2xdTXM3,6.322800,-0.365680,-4.918161,-3.823232,-5.706920,-0.875162,1.567523,-1.347518,-1.717427,1.626777,...,-0.700429,1.121223,0.806392,0.487863,-1.078968,4.874758,1.000130,0.497940,-1.314523,-0.334021
WHWymJu6zNZi,5.468294,-3.985252,1.542399,0.104717,-0.529033,-2.951971,0.786361,0.794939,-2.772054,-2.408742,...,-1.317848,-0.099000,-0.702003,-0.529044,1.634401,-0.246858,-0.448421,-1.545799,0.016255,-0.905541
4PAQp1M6EyAo,0.447190,0.733000,3.051331,1.763902,1.932819,6.451160,-3.356867,1.946047,-1.165669,-2.876914,...,2.705387,-0.852947,1.701029,0.686384,-0.031653,1.806995,0.973496,-1.922737,0.069176,-2.304174
obEacy4Of68I,-9.149799,-2.044626,0.519359,-2.703922,-4.444127,2.916929,1.036385,0.080352,-0.148121,0.384170,...,0.395511,1.584922,1.185720,-0.410873,1.216775,-0.065100,0.721908,-0.158146,0.247976,-2.405702
s7WzzDcmDOhF,0.812814,-1.933265,1.834524,0.383157,-0.512341,2.972215,3.151071,3.419853,0.283723,-2.997841,...,-2.318394,2.920521,-1.609283,-0.852692,-0.133424,-1.059160,1.188180,-0.155662,2.718990,0.827381


In [77]:
test_func_pca.head()

,fmri_pca_1,fmri_pca_2,fmri_pca_3,fmri_pca_4,fmri_pca_5,fmri_pca_6,fmri_pca_7,fmri_pca_8,fmri_pca_9,fmri_pca_10,...,fmri_pca_41,fmri_pca_42,fmri_pca_43,fmri_pca_44,fmri_pca_45,fmri_pca_46,fmri_pca_47,fmri_pca_48,fmri_pca_49,fmri_pca_50
participant_id,,,,,,,,,,,,,,,,,,,,,
Cfwaf5FX7jWK,-0.241573,0.798620,-3.338365,-2.945730,0.935485,2.380042,-2.108530,0.727716,-1.825227,-2.339715,...,0.180244,-1.594318,-0.774946,-1.808223,0.611929,-4.189920,1.562126,0.011852,3.471165,2.008843
vhGrzmvA3Hjq,-6.393446,-2.606063,-1.673228,-4.574098,2.661956,-2.814177,-0.420013,0.547968,0.260094,-2.889240,...,-0.308447,0.014417,-1.092795,1.585590,-2.695314,3.748906,1.680684,-0.173057,-0.311072,-0.324432
ULliyEXjy4OV,-1.601297,0.157037,0.352906,6.263209,-2.054009,-1.135146,2.228638,-0.690495,2.496552,-0.638298,...,-0.434658,0.298066,-1.775209,-0.597397,0.671241,-1.105910,0.044946,-0.193182,-1.217737,3.561735
LZfeAb1xMtql,-1.571668,-2.720598,-2.393926,0.868357,0.737472,-0.501450,2.458480,-0.002225,0.104631,-1.108050,...,-1.315546,0.578891,-0.388128,0.442122,0.001584,-0.590576,1.628386,0.423327,-1.035141,1.569984
EnFOUv0YK1RG,2.889024,-1.047001,-2.082233,-0.431934,-2.277369,-1.308704,-0.252739,0.279537,1.585714,0.285816,...,0.260851,-0.449771,0.748327,0.389085,0.948853,2.090336,2.259653,-1.170112,-2.119113,-1.464002


## Interpolate missing values with mean for quant and cat dfs

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer

def impute_with_regression(df, target_col):
    """Impute missing values in target_col of df using linear regression."""
    df = df.copy()
    train_df = df.dropna(subset=[target_col])
    if train_df.empty:
        return df  # skip if no data to train on
    X_train = train_df.drop(columns=[target_col])
    y_train = train_df[target_col]
    X_train_imp = SimpleImputer(strategy='mean').fit_transform(X_train)
    reg = LinearRegression().fit(X_train_imp, y_train)

    missing_mask = df[target_col].isna()
    X_missing = df.loc[missing_mask].drop(columns=[target_col])
    X_missing_imp = SimpleImputer(strategy='mean').fit_transform(X_missing)
    df.loc[missing_mask, target_col] = reg.predict(X_missing_imp)
    return df

# Function to process any dataframe (quant or cat)
def process_df(df, name='df'):
    missing = df.isnull().sum()
    for col in missing[missing > 0].index:
        if missing[col] < 50:
            df[col] = df[col].fillna(df[col].mean())
        else:
            print(f"Using regression to impute '{col}' in {name}")
            df = impute_with_regression(df, col)
    return df

# Apply to all four
train_quant = process_df(train_quant, 'train_quant')
train_cat = process_df(train_cat, 'train_cat')
test_quant = process_df(test_quant, 'test_quant')
test_cat = process_df(test_cat, 'test_cat')

Using regression to impute 'MRI_Track_Age_at_Scan' in train_quant
Using regression to impute 'PreInt_Demos_Fam_Child_Race' in train_cat
Using regression to impute 'Barratt_Barratt_P2_Edu' in train_cat
Using regression to impute 'Barratt_Barratt_P2_Occ' in train_cat


In [6]:
# Check missing values *before* merging
missing = test_cat.isnull().sum()
print("Missing values in train_quant:\n", missing[missing > 0])

Missing values in train_quant:
 Series([], dtype: int64)


In [80]:
train_quant.head()

,EHQ_EHQ_Total,ColorVision_CV_Score,APQ_P_APQ_P_CP,APQ_P_APQ_P_ID,APQ_P_APQ_P_INV,APQ_P_APQ_P_OPD,APQ_P_APQ_P_PM,APQ_P_APQ_P_PP,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing,SDQ_SDQ_Peer_Problems,SDQ_SDQ_Prosocial,MRI_Track_Age_at_Scan
participant_id,,,,,,,,,,,,,,,,,,
00aIpNTbG5uh,100.00,13.0,3.0,15.0,44.0,14.0,20.0,27.0,3.0,17.0,4.0,11.0,5.0,8.0,6.0,2.0,9.0,14.274127
00fV0OyyoLfw,92.27,14.0,3.0,12.0,35.0,25.0,28.0,30.0,5.0,20.0,4.0,13.0,5.0,8.0,7.0,3.0,8.0,14.278626
04X1eiS79T4B,86.67,14.0,3.0,21.0,37.0,18.0,26.0,28.0,3.0,24.0,7.0,10.0,10.0,7.0,14.0,7.0,7.0,13.463381
05ocQutkURd6,93.34,14.0,3.0,11.0,42.0,15.0,20.0,28.0,0.0,5.0,0.0,3.0,0.0,3.0,2.0,2.0,6.0,9.572553
06YUNBA9ZRLq,0.00,14.0,8.0,12.0,35.0,22.0,12.0,24.0,6.0,23.0,7.0,15.0,8.0,9.0,8.0,1.0,4.0,6.654574


In [7]:
# make participant_id a column
train_quant = train_quant.reset_index()
train_cat = train_cat.reset_index()
train_soln = train_soln.reset_index()
test_quant = test_quant.reset_index()
test_cat = test_cat.reset_index()

# Merge everything on participant_id
X_full = train_quant.merge(train_cat, on='participant_id').merge(train_func_pca, on='participant_id')
X_full_test = test_quant.merge(test_cat, on='participant_id').merge(test_func_pca, on='participant_id')

# Target labels for model training
y = train_soln[['participant_id', 'ADHD_Outcome', 'Sex_F']]

# Align rows across X and y
X_full = X_full.merge(y, on='participant_id')
X_full = X_full.drop(columns=['ADHD_Outcome', 'Sex_F'])
X_full.set_index("participant_id", inplace=True)
X_full_test.set_index("participant_id", inplace=True)
X_full_test.head()

,EHQ_EHQ_Total,ColorVision_CV_Score,APQ_P_APQ_P_CP,APQ_P_APQ_P_ID,APQ_P_APQ_P_INV,APQ_P_APQ_P_OPD,APQ_P_APQ_P_PM,APQ_P_APQ_P_PP,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,...,fmri_pca_41,fmri_pca_42,fmri_pca_43,fmri_pca_44,fmri_pca_45,fmri_pca_46,fmri_pca_47,fmri_pca_48,fmri_pca_49,fmri_pca_50
participant_id,,,,,,,,,,,,,,,,,,,,,
Cfwaf5FX7jWK,60.03,14.0,5.0,16.0,41.0,19.0,11.0,26.0,2.0,12.0,...,-2.354827,-0.245859,-2.495723,-0.612933,0.316750,0.435766,-5.143142,1.376251,-0.692517,-0.919421
vhGrzmvA3Hjq,86.71,12.0,3.0,13.0,43.0,18.0,15.0,28.0,2.0,16.0,...,-0.657587,0.792164,-1.406735,3.491856,3.084778,-0.259822,0.500463,-1.507016,1.506525,-0.459680
ULliyEXjy4OV,26.68,13.0,3.0,14.0,36.0,16.0,14.0,25.0,1.0,7.0,...,-0.273305,-0.529575,-1.577311,-0.063774,2.484738,-1.782543,-3.936662,0.881311,0.411183,-1.568857
LZfeAb1xMtql,93.38,13.0,3.0,19.0,41.0,17.0,18.0,27.0,4.0,15.0,...,-1.906388,-1.247441,0.946841,-0.119457,3.118051,-0.334814,0.302098,-0.204796,1.547164,-0.057709
EnFOUv0YK1RG,-93.38,14.0,3.0,13.0,42.0,19.0,16.0,28.0,2.0,18.0,...,-0.532027,1.331892,-1.102122,1.359646,-0.757888,-0.122841,0.668749,0.207636,0.730948,1.833062


In [18]:
train_soln.head()

,participant_id,ADHD_Outcome,Sex_F
0,UmrK0vMLopoR,1,1
1,CPaeQkhcjg7d,1,0
2,Nb4EetVPm3gs,1,0
3,p4vPhVu91o4b,1,1
4,M09PXs7arQ5E,1,1


In [30]:
from sklearn.preprocessing import StandardScaler

# Fit scaler only on training data
scaler = StandardScaler()
scaler.fit(X_full)

# Transform both train and test
X_scaled = pd.DataFrame(scaler.transform(X_full), columns=X_full.columns, index=X_full.index)
X_scaled_test = pd.DataFrame(scaler.transform(X_full_test), columns=X_full_test.columns, index=X_full_test.index)
X_scaled_test.head()

,EHQ_EHQ_Total,ColorVision_CV_Score,APQ_P_APQ_P_CP,APQ_P_APQ_P_ID,APQ_P_APQ_P_INV,APQ_P_APQ_P_OPD,APQ_P_APQ_P_PM,APQ_P_APQ_P_PP,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,...,fmri_pca_41,fmri_pca_42,fmri_pca_43,fmri_pca_44,fmri_pca_45,fmri_pca_46,fmri_pca_47,fmri_pca_48,fmri_pca_49,fmri_pca_50
participant_id,,,,,,,,,,,,,,,,,,,,,
Cfwaf5FX7jWK,0.010418,0.277591,0.892293,0.744989,0.254265,0.312612,-1.085593,0.165030,-0.037104,-0.032888,...,-1.519326,-0.159310,-1.621435,-0.404216,0.212295,0.293837,-3.514891,0.941870,-0.475583,-0.642889
vhGrzmvA3Hjq,0.549880,-0.678512,-0.619194,-0.094347,0.666930,0.011048,-0.304174,0.823507,-0.037104,0.583406,...,-0.424273,0.513302,-0.913935,2.302803,2.067512,-0.175198,0.342023,-1.031363,1.034598,-0.321423
ULliyEXjy4OV,-0.663909,-0.200460,-0.619194,0.185432,-0.777397,-0.592079,-0.499529,-0.164208,-0.533472,-0.803255,...,-0.176335,-0.343151,-1.024756,-0.042057,1.665347,-1.201971,-2.690367,0.603147,0.282378,-1.096996
LZfeAb1xMtql,0.684745,-0.200460,-0.619194,1.584325,0.254265,-0.290515,0.281890,0.494269,0.955631,0.429333,...,-1.229995,-0.808309,0.615148,-0.078779,2.089813,-0.225765,0.206458,-0.140157,1.062507,-0.040352
EnFOUv0YK1RG,-3.091485,0.277591,-0.619194,-0.094347,0.460597,0.312612,-0.108820,0.823507,-0.037104,0.891553,...,-0.343262,0.863032,-0.716032,0.896657,-0.507960,-0.082832,0.457032,0.142101,0.501975,1.281737


## XGBoost For modelling

In [38]:
from sklearn.cluster import KMeans
import pandas as pd

X_train = X_scaled
X_test = X_scaled_test

# Cluster the training set
kmeans = KMeans(n_clusters=7, random_state=42)
train_clusters = kmeans.fit_predict(X_scaled)

# Add cluster labels
X_train_df['cluster'] = train_clusters

# Group by cluster and compute mean feature values
cluster_means = X_train_df.groupby('cluster').mean()

# Get feature importance by variance across cluster means
feature_variances = cluster_means.var()

# Select top 20 features
top_features = feature_variances.sort_values(ascending=False).index[:20]
print(f"Top 20 features:\n{top_features}")

Top 20 selected features:
Index(['SDQ_SDQ_Difficulties_Total', 'fmri_pca_28', 'SDQ_SDQ_Internalizing',
       'SDQ_SDQ_Externalizing', 'MRI_Track_Age_at_Scan',
       'SDQ_SDQ_Generating_Impact', 'SDQ_SDQ_Hyperactivity',
       'SDQ_SDQ_Emotional_Problems', 'SDQ_SDQ_Conduct_Problems',
       'SDQ_SDQ_Peer_Problems', 'MRI_Track_Scan_Location', 'APQ_P_APQ_P_PM',
       'Basic_Demos_Study_Site', 'fmri_pca_44', 'Basic_Demos_Enroll_Year',
       'fmri_pca_25', 'Barratt_Barratt_P2_Occ', 'APQ_P_APQ_P_OPD',
       'fmri_pca_32', 'fmri_pca_8'],
      dtype='object')


InvalidIndexError: (slice(None, None, None), Index(['SDQ_SDQ_Difficulties_Total', 'fmri_pca_28', 'SDQ_SDQ_Internalizing',
       'SDQ_SDQ_Externalizing', 'MRI_Track_Age_at_Scan',
       'SDQ_SDQ_Generating_Impact', 'SDQ_SDQ_Hyperactivity',
       'SDQ_SDQ_Emotional_Problems', 'SDQ_SDQ_Conduct_Problems',
       'SDQ_SDQ_Peer_Problems', 'MRI_Track_Scan_Location', 'APQ_P_APQ_P_PM',
       'Basic_Demos_Study_Site', 'fmri_pca_44', 'Basic_Demos_Enroll_Year',
       'fmri_pca_25', 'Barratt_Barratt_P2_Occ', 'APQ_P_APQ_P_OPD',
       'fmri_pca_32', 'fmri_pca_8'],
      dtype='object'))

In [35]:
# Check cross-validation scores (NOT the overfitted train scores)
print(f"\nCross-Validation F1 Score for Sex_F model: {sex_grid_search.best_score_:.4f}")
print(f"Cross-Validation F1 Score for ADHD_Outcome model: {adhd_grid_search.best_score_:.4f}")

# Predict on test set
sex_test_preds = best_sex_model.predict(X_scaled_test)
adhd_test_preds = best_adhd_model.predict(X_scaled_test)

# If you want to make a nice DataFrame for submission or analysis:
submission = pd.DataFrame({
    'participant_id': X_full_test.index,
    'Sex_F': sex_test_preds,
    'ADHD_Outcome': adhd_test_preds
})

submission.head()


Cross-Validation F1 Score for Sex_F model: 0.5756
Cross-Validation F1 Score for ADHD_Outcome model: 0.6013


,participant_id,Sex_F,ADHD_Outcome
0,Cfwaf5FX7jWK,1,1
1,vhGrzmvA3Hjq,1,1
2,ULliyEXjy4OV,0,1
3,LZfeAb1xMtql,0,1
4,EnFOUv0YK1RG,1,0


In [36]:
submission.to_csv('submission.csv', index=False)